# 03: Export NAIP

This notebook exports NAIP imagery from GEE for use in predictive mapping with a CNN model.

In [1]:
import ee

from naip_cnn.config import BANDS, CRS

ee.Initialize()

Below, we'll define the year, image size, and region that we want to generate a predicted map across.

In [5]:
SHAPE = (150, 150)
NAIP_YEAR = 2016
BBOX = ee.Geometry.Polygon(
    [
        [
            [-119.37239004502233, 44.48079613290612],
            [-118.57725454697545, 44.48079613290612],
            [-118.57725454697545, 44.81785572318615],
            [-119.37239004502233, 44.81785572318615],
        ]
    ],
    None,
    False,
)

Using those parameters, we can create a mosaic of NAIP imagery.

In [7]:
naip_mosaic = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(BBOX)
    .filterDate(str(NAIP_YEAR), str(NAIP_YEAR + 1))
    .mosaic()
    .select(BANDS)
    .reproject(CRS)
    .uint8()
)

Now we can export the TFRecords to Google Drive, where we can grab them once the task is complete. The status can be tracked using `task.status()` or in the [Earth Engine Task Manager](https://code.earthengine.google.com/tasks).

In [8]:
task = ee.batch.Export.image.toDrive(
    image=naip_mosaic,
    description=f"naip_{NAIP_YEAR}",
    region=BBOX,
    scale=1,
    fileFormat="TFRecord",
    maxPixels=1e13,
    formatOptions={
        "patchDimensions": SHAPE,
        "compressed": True,
        "maxFileSize": 1e9,
    },
)

task.start()